In [1]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
#from pyspark.sql import SQLContext
from pyspark import SparkConf,SparkContext
from pyspark.sql.types import IntegerType, StringType
#from pyspark.sql.functions import udf
import pyspark.sql.functions as funct
import os

path = "C:/Users/scohe/Documents/GitHub/MasterIASD_Projet_CCF/prototype/input/"
partition_number = 1
#conf.set('spark.driver.host', '127.0.0.1')
#os.environ['PYSPARK_PYTHON'] = '/Users/ccompain/.pyenv/versions/miniconda3-latest/bin/python' # Needs to be explicitly provided as env. Otherwise workers run Python 2.7
#os.environ['PYSPARK_DRIVER_PYTHON'] = 'jupyter'

#sc = SparkContext(conf=conf)
spark = SparkSession.builder\
.master("local")\
.appName("pyspark-shell")\
.config('spark.driver.host', '127.0.0.1')\
.config("spark.logConf", "true").getOrCreate() #.config("spark.driver.memory", "16g")
#sqlContext = SQLContext(spark)
#sqlContext.setConf("spark.sql.shuffle.partitions", "300")
#sqlContext.setConf("spark.default.parallelism", "300")
spark.sparkContext.setLogLevel("WARN") 

In [4]:
r =  spark.read\
            .format("csv").option("header", "true")\
            .option("inferSchema", "true")\
            .load(path +"simple_2_graphs.csv").toDF("N1","N2").coalesce(partition_number)
#r=spark.createDataFrame([("A","B"),("B","C"),("B","D"),("D","E"),("F","G"),("G","H")]).toDF("N1","N2") 
#r.printSchema()
r.show()
#r.collect()

+---+---+
| N1| N2|
+---+---+
|  1|  3|
|  2|  3|
|  2|  4|
|  2|  5|
|  3|  4|
|  4|  5|
|  4|  6|
|  5|  7|
|  5|  8|
|  7|  8|
| 13| 14|
| 13| 15|
| 13|  9|
| 14| 15|
| 15| 10|
|  9| 10|
+---+---+



In [2]:
log4jLogger = spark._jvm.org.apache.log4j
LOGGER = log4jLogger.LogManager.getLogger(__name__)


In [22]:
new_pair_flag = True
iteration = 0
dedupJob = r


In [17]:
mapJob = dedupJob.union(dedupJob.select('N2', 'N1')).coalesce(partition_number).persist()
minDF=mapJob.groupBy('N1').agg(funct.min(mapJob['N2']).alias('minN')).where('min(N2)<N1').persist()
supplDF=mapJob.join(minDF, "N1").where("minN<>N2").select('N2','minN').withColumnRenamed('N2','N1').withColumnRenamed('minN','N2').persist()
reduceJob = supplDF.union(minDF).coalesce(partition_number).persist()
print(reduceJob.rdd.glom().getNumPartitions())

1


In [23]:
while new_pair_flag:
    iteration += 1
    newPair = False
    newPairs = 0

    # CCF-iterate (MAP)
    mapJob = dedupJob.union(dedupJob.select('N2', 'N1')).coalesce(partition_number).persist()
  #  print("Map "+str(iteration))
  #  print(mapJob.show())

    # CCF-iterate (REDUCE)
    minDF=mapJob.groupBy('N1').agg(funct.min(mapJob['N2']).alias('minN')).where('min(N2)<N1').persist()
  #  print("minDF "+str(iteration))
  #  print(minDF.show())
    supplDF=mapJob.join(minDF, "N1").where("minN<>N2").select('N2','minN').withColumnRenamed('N2','N1').withColumnRenamed('minN','N2').persist()
  #  print("supplDF "+str(iteration))
  #  print(supplDF.show())
    newPairs=supplDF.count()
    reduceJob = supplDF.union(minDF).coalesce(partition_number).persist()
  #  print("reduceJob "+str(iteration))
  #  print(reduceJob.show())
    
    # CCF-dedup 
    dedupJob = reduceJob.distinct().persist() #.orderBy("N1", "N2")
  #  print("dedupJob "+str(iteration))
  #  print(dedupJob.show())

    # Force the RDD evalusation
#    tmp = dedupJob.count()

    # Prepare next iteration
    #mapJob = dedupJob
    new_pair_flag = bool(newPairs)

    #LOGGER.warn("Iteration :"+str(iteration)+" --- "+" newPair : "+str(new_pair_flag))
    print("Iteration :"+str(iteration)+" --- "+" newPairs : "+str(newPairs))


Iteration :1 ---  newPairs : 16
Iteration :2 ---  newPairs : 24
Iteration :3 ---  newPairs : 10
Iteration :4 ---  newPairs : 0


In [50]:
#count1 = funct.udf(lambda s: s+1, IntegerType())
#_ = spark.udf.register("count1", count1)

result= dedupJob.groupBy('N2').count().orderBy('count',ascending=False).withColumnRenamed('N2','Group').withColumnRenamed('count','otherComponentsCount') #.withColumn('nbComponents',count1('count')).select('N2','NbComponents').orderBy('NbComponents',ascending=False)
print(result.show())


+-----+--------------------+
|Group|otherComponentsCount|
+-----+--------------------+
|    1|                   7|
|    9|                   4|
+-----+--------------------+

None
